In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install zoofs
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd


  Preparing metadata (setup.py) ... done
  Created wheel for zoofs: filename=zoofs-0.1.26-py3-none-any.whl size=23599 sha256=70b7ae4593ab1b8d512b8213762c59dfce9bf53d61e0e7c202323cc8d490f493
  Stored in directory: /root/.cache/pip/wheels/51/47/cc/500199ac6341b98830297864a204c9fb3378e015fafeb8423b
Successfully built zoofs


In [3]:
import os
train_file = '/content/drive/MyDrive/StormSurgeFS/StormVNTraining.csv'
test_file = '/content/drive/MyDrive/StormSurgeFS/StormVNTesting.csv'
train = pd.read_csv(train_file)
test = pd.read_csv(test_file)
type(train)

pandas.core.frame.DataFrame

In [5]:
from re import X
from sklearn.preprocessing import StandardScaler
X_train=train.drop('output', axis=1)
y_train=train['output']
X_test=test.drop('output', axis=1)
y_test=test['output']
#X_train_norm = (X_train-X_train.min())/(X_train.max()-X_train.min())
#X_test_norm = (X_test-X_test.min())/(X_test.max()-X_test.min())
#type(X_train_norm)

In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
# ANN
ann_model = MLPRegressor(random_state=42)
ann_scores = cross_val_score(ann_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print(ann_scores)
param_grid_ann = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 100)],
    'activation': ['identity', 'logistic', 'tanh', 'relu'],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'alpha': [0.0001, 0.001, 0.01, 0.1],
    'learning_rate': ['constant', 'invscaling', 'adaptive']
}

grid_search_ann = GridSearchCV(ann_model, param_grid_ann, scoring='neg_mean_squared_error', cv=5, n_jobs=-1, verbose=1)
grid_search_ann.fit(X_train, y_train)
test_score_ann = grid_search_ann.best_estimator_.score(X_train, y_train)

print(f"Best hidden_layer_sizes value: {grid_search_ann.best_params_['hidden_layer_sizes']}")
print(f"Best activation value: {grid_search_ann.best_params_['activation']}")
print(f"Best solver value: {grid_search_ann.best_params_['solver']}")
print(f"Best alpha value: {grid_search_ann.best_params_['alpha']}")
print(f"Best learning_rate value: {grid_search_ann.best_params_['learning_rate']}")
print(f"Best negative mean squared error: {grid_search_ann.best_score_}")

[-0.08561528 -0.19024902 -0.07115309 -0.17655118 -0.10522793]
Fitting 5 folds for each of 576 candidates, totalling 2880 fits
Best hidden_layer_sizes value: (100, 100)
Best activation value: relu
Best solver value: adam
Best alpha value: 0.001
Best learning_rate value: constant
Best negative mean squared error: -0.11087288164929561


In [7]:
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from math import sqrt
from scipy.stats.stats import pearsonr
#reg = MLPRegressor(random_state=0,  activation='relu', solver='adam',  learning_rate='constant', alpha=0.001, early_stopping=True, hidden_layer_sizes=(100,100), max_iter=10000, momentum=0.001)
#reg=KNeighborsRegressor(n_neighbors=7)
#reg=SVR(kernel='rbf',epsilon=0.01)
#reg = RandomForestRegressor(n_estimators=50, random_state=0, oob_score=True)
from sklearn.ensemble import GradientBoostingRegressor
reg = GradientBoostingRegressor(random_state=0,  n_estimators=150)
reg.fit(X_train, y_train)
y_pred=reg.predict(X_test)
error=sqrt(mean_squared_error(y_pred, y_test))/(max(y_test)-min(y_test))
#r2=r2_score(y_pred, y_test)
nr2=pearsonr(y_pred, y_test)
print(nr2)
print('Model accuracy score: {0:0.4f}'.format(error))

PearsonRResult(statistic=0.7787483276477778, pvalue=7.594803273533173e-17)
Model accuracy score: 0.1624


<ipython-input-7-8031f851b85f>:6: DeprecationWarning: Please import `pearsonr` from the `scipy.stats` namespace; the `scipy.stats.stats` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.stats.stats import pearsonr


In [18]:
from zoofs import GeneticOptimization
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score, mean_squared_error

from scipy.stats.stats import pearsonr
def objective_function_topass(model,X_train, y_train, X_valid, y_valid):
    model.fit(X_train,y_train)
    P=mean_squared_error(y_valid,model.predict(X_valid))
    return P
# create object of algorithm
algo_object=GeneticOptimization(objective_function_topass,n_iteration=10,
                            population_size=50,selective_pressure=2.4,elitism=3,
                            mutation_rate=0.1,minimize=True)
from sklearn.ensemble import GradientBoostingRegressor
#reg = GradientBoostingRegressor(random_state=0)
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
#reg = GradientBoostingRegressor(random_state=0,  n_estimators=150)
#reg = MLPRegressor(random_state=0,  activation='relu', solver='adam',  learning_rate='constant', alpha=0.001, early_stopping=True, hidden_layer_sizes=(100,100), max_iter=10000, momentum=0.001)
#reg=KNeighborsRegressor(n_neighbors=7)
#reg=SVR(kernel='rbf',epsilon=0.01)
regressor = RandomForestRegressor(n_estimators=50, random_state=0, oob_score=True)
# fit the algorithm
algo_object.fit(reg,X_train, y_train, X_test, y_test,verbose=True)
feat = algo_object.best_feature_list
#feat
reg.fit(X_train[feat], y_train)
y_pred=reg.predict(X_test[feat])
error=sqrt(mean_squared_error(y_pred, y_test))/(max(y_test)-min(y_test))
#r2=r2_score(y_pred, y_test)
nr2=pearsonr(y_pred, y_test)
print(nr2)
print('LightGBM Model accuracy score: {0:0.4f}'.format(error))
feat

<ipython-input-18-1fd98fcb9021>:6: DeprecationWarning: Please import `pearsonr` from the `scipy.stats` namespace; the `scipy.stats.stats` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.stats.stats import pearsonr
 [ 2025-03-04 08:03:04,377 ] Finished iteration #0 with objective value 0.09285320495568249. Current best value is 0.09285320495568249 
 [ 2025-03-04 08:03:04,513 ] Finished iteration #1 with objective value 0.09244933630189175. Current best value is 0.09244933630189175 
 [ 2025-03-04 08:03:04,589 ] Finished iteration #2 with objective value 0.09526255744138624. Current best value is 0.09244933630189175 
 [ 2025-03-04 08:03:04,639 ] Finished iteration #3 with objective value 0.09244933630189175. Current best value is 0.09244933630189175 
 [ 2025-03-04 08:03:04,661 ] Finished iteration #4 with objective value 0.09244933630189175. Current best value is 0.09244933630189175 
 [ 2025-03-04 08:03:04,682 ] Finished iteration #5 with objective value 0.0924493

PearsonRResult(statistic=0.7657098823659145, pvalue=5.014784116449305e-16)
LightGBM Model accuracy score: 0.1679


['WS', 'WD', 'DSLP', 'LG', 'LT', 'CAP', 'HWS', 'SS']

In [43]:
from zoofs import GeneticOptimization
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from math import sqrt
# define your own objective function, make sure the function receives four parameters,
#  fit your model and return the objective value !
def objective_function_topass(model,X_train, y_train, X_valid, y_valid, **kwargs):
    features = kwargs.get('features', X_train.columns.tolist())
    #features = kwargs.get('features', X_train.columns.tolist())
    # Re-initialize the model with the correct number of input features
    model = keras.Sequential([
        layers.Input(shape=(len(features),)),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(16, activation='relu'),
        layers.Dense(8, activation='relu'),
        layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    P=mean_squared_error(y_valid,model.predict(X_valid))
    return P
# create object of algorithm
#algo_object=GeneticOptimization(objective_function_topass,n_iteration=15,
#                            population_size=20,selective_pressure=2.4,elitism=2,
#                            mutation_rate=0.1,minimize=True)
#from sklearn.ensemble import GradientBoostingRegressor
algo_object=GeneticOptimization(objective_function_topass,n_iteration=15,
                            population_size=20,selective_pressure=2.4,elitism=2,
                            mutation_rate=0.1,minimize=True,kwargs = {'features': X_train.columns.tolist()})

#from lightgbm import LGBMRegressor
#reg = LGBMRegressor(boosting_type = 'gbdt', n_estimators=100)
model = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),        # Input layer
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),  # Hidden layer with 32 neurons and ReLU activation
    layers.Dense(16, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(1)                  # Output layer with a single neuron (for regression)
])
# fit the algorithm
model.compile(optimizer='adam', loss='mean_squared_error')
algo_object.fit(model, X_train, y_train, X_test, y_test,verbose=True)
feat = algo_object.best_feature_list
feat
#reg.fit(X_train[feat], y_train)
#y_pred=reg.predict(X_test[feat])
#error=sqrt(mean_squared_error(y_test, y_pred))/(max(y_test)-min(y_test))
#r2=sqrt(r2_score(y_pred, y_test))
#print('LightGBM Model accuracy score: {0:0.4f}'.format(r2))
#print('LightGBM Model accuracy score: {0:0.4f}'.format(error))


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step


 [ 2025-03-04 02:34:10,990 ] Finished iteration #0 with objective value 0.2383186215945522. Current best value is 0.2383186215945522 


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


...

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


 [ 2025-03-04 02:34:18,596 ] Finished iteration #4 with objective value 0.5307310617483277. Current best value is 0.1765574116631038 
 [ 2025-03-04 02:34:18,610 ] Finished iteration #5 with objective value 0.4565102956719839. Current best value is 0.1765574116631038 


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step


 [ 2025-03-04 02:34:18,886 ] Finished iteration #6 with objective value 0.5307310617483277. Current best value is 0.1765574116631038 
 [ 2025-03-04 02:34:18,901 ] Finished iteration #7 with objective value 0.5307310617483277. Current best value is 0.1765574116631038 
 [ 2025-03-04 02:34:18,912 ] Finished iteration #8 with objective value 0.4565102956719839. Current best value is 0.1765574116631038 
 [ 2025-03-04 02:34:18,925 ] Finished iteration #9 with objective value 0.5307310617483277. Current best value is 0.1765574116631038 
 [ 2025-03-04 02:34:18,940 ] Finished iteration #10 with objective value 0.5307310617483277. Current best value is 0.1765574116631038 
 [ 2025-03-04 02:34:18,955 ] Finished iteration #11 with objective value 0.5307310617483277. Current best value is 0.1765574116631038 
 [ 2025-03-04 02:34:18,970 ] Finished iteration #12 with objective value 0.5307310617483277. Current best value is 0.1765574116631038 
 [ 2025-03-04 02:34:18,984 ] Finished iteration #13 with ob

['WS', 'SSL', 'LT', 'SS']

In [44]:
feat


['WS', 'SSL', 'LT', 'SS']

In [53]:
y_train_new=y_train
X_train_new=X_train[['WD', 'SLP', 'SSL', 'LG', 'LT', 'CAP', 'SS']]
X_test_new=X_test[['WD', 'SLP', 'SSL', 'LG', 'LT', 'CAP', 'SS']]
y_test_new=y_test
X_test_new.shape
y_test_new.shape

(77,)

In [54]:
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras import layers
from keras.layers import Dense, Conv1D, Flatten
model = keras.Sequential([
    layers.Input(shape=(X_train_new.shape[1],)),        # Input layer
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),  # Hidden layer with 32 neurons and ReLU activation
    layers.Dense(16, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(1)                  # Output layer with a single neuron (for regression)
])
model.compile(loss="mse", optimizer="adam")

In [57]:
model.fit(X_train_new, y_train_new, batch_size=10,epochs=200, verbose=0)

In [58]:
import math

y_pred = model.predict(X_test_new)
# Reshape y_pred to 1-dimensional array
y_pred = y_pred.reshape(-1)
#y_pred = y_pred[:len(y_test_new)]
mse = math.sqrt(mean_squared_error(y_test, y_pred))/(max(y_test)-min(y_test))
nr2=pearsonr(y_pred, y_test)
print(f"MSE: {mse:.4f}")
print(nr2)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
MSE: 0.1746
PearsonRResult(statistic=0.7433753812739083, pvalue=9.730705864105697e-15)
